# Comparing 2016 and 2021 Data

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

## 1. Data
### 1.1 Load Data

In [2]:
df_2016 = pd.read_excel("processed/2016_data_good.xlsx")
df_2016.head()

,S/T code,S/T name,GCCSA code,GCCSA name,SA4 code,SA4 name,SA3 code,SA3 name,SA2 code,SA2 name,...,45–49,50–54,55–59,60–64,65–69,70–74,75–79,80–84,85 and over,Total Persons
0,1.0,New South Wales,1RNSW,Rest of NSW,101.0,Capital Region,10102.0,Queanbeyan,11007.0,Braidwood,...,291.0,294.0,393.0,315.0,336.0,253.0,159.0,71.0,70.0,3950.0
1,1.0,New South Wales,1RNSW,Rest of NSW,101.0,Capital Region,10102.0,Queanbeyan,11008.0,Karabar,...,631.0,579.0,573.0,481.0,393.0,270.0,215.0,99.0,80.0,8531.0
2,1.0,New South Wales,1RNSW,Rest of NSW,101.0,Capital Region,10102.0,Queanbeyan,11009.0,Queanbeyan,...,721.0,704.0,604.0,530.0,484.0,341.0,278.0,243.0,305.0,11230.0
3,1.0,New South Wales,1RNSW,Rest of NSW,101.0,Capital Region,10102.0,Queanbeyan,11010.0,Queanbeyan - East,...,365.0,349.0,339.0,239.0,180.0,143.0,95.0,54.0,40.0,4970.0
4,1.0,New South Wales,1RNSW,Rest of NSW,101.0,Capital Region,10102.0,Queanbeyan,11011.0,Queanbeyan Region,...,1504.0,1425.0,1303.0,1098.0,891.0,517.0,286.0,122.0,83.0,17291.0


In [3]:
df_2021 = pd.read_csv("processed/SA2PopulationData.csv")
df_2021.head()

,SA2_CODE_2021,SA2_5DIG21,SA2_5DIG21.1,SA2_CODE_2016,SA2_CODE_2016.1,SA2_5DIG16,SA2_5DIG16.1,SA2_NAME_2021,SA2_AREA,ERP_P_202021,ERP_P_202021 (%),ERP_P_202021 (%).1,ERP_212021,ERP_M_202021,ERP_M_202021 (%),ERP_M_202021 (%).1,ERP_F_202021,ERP_F_202021 (%),ERP_F_202021 (%).1
0,101021007,11007,11007,101021007,101021007,11007,11007,Braidwood,3418.3525,4330.0,0.016856,0.016856,1.3,2248.0,0.017632,0.017632,2082.0,0.016092,0.016092
1,101021008,11008,11008,101021008,101021008,11008,11008,Karabar,6.9825,8546.0,0.033268,0.033268,1223.9,4324.0,0.033915,0.033915,4222.0,0.032631,0.032631
2,101021009,11009,11009,101021009,101021009,11009,11009,Queanbeyan,4.7620,11370.0,0.044262,0.044262,2387.7,5788.0,0.045398,0.045398,5582.0,0.043143,0.043143
3,101021010,11010,11010,101021010,101021010,11010,11010,Queanbeyan - East,13.0032,5093.0,0.019826,0.019826,391.7,2671.0,0.020950,0.020950,2422.0,0.018719,0.018719
4,101021012,11012,11012,101021012,101021012,11012,11012,Queanbeyan West - Jerrabomberra,13.6748,12743.0,0.049607,0.049607,931.9,6387.0,0.050096,0.050096,6356.0,0.049125,0.049125


In [4]:
df_2021["SA2_AREA"].isna().value_counts()

SA2_AREA
False    2157
True       19
Name: count, dtype: int64

### 1.2 Preprocess Data
Format specific columns in both DataFrames to merge them later.

#### 1.2.1 Population Data 2016
Drop all rows with NaN values and replace names to match the one from the 2021 data.

In [5]:
df_2016 = df_2016.dropna(ignore_index=True)

replace_dct = {
    "SA2 code":"SA2_5DIG16",
    "SA2 name":"SA2_NAME_2016",
    "Total Persons":"NR_OF_PEOPLE_2016"
}

df_2016.rename(replace_dct,axis=1,inplace=True)

Convert 5 digit SA2 code to string, and create 9 digit SA2 code.

In [6]:
df_2016["SA2_CODE_2016"] = [str(round(row["SA3 code"]))[:]+str(round(row["SA2_5DIG16"]))[1:] for _,row in df_2016.iterrows()]
df_2016["SA2_5DIG16"] = [str(round(i)) for i in df_2016["SA2_5DIG16"]]


Select only the interesting columns.

In [7]:
df_2016 = df_2016[["SA2_CODE_2016","SA2_5DIG16","SA2_NAME_2016","NR_OF_PEOPLE_2016"]]
df_2016.head()

,SA2_CODE_2016,SA2_5DIG16,SA2_NAME_2016,NR_OF_PEOPLE_2016
0,101021007,11007,Braidwood,3950.0
1,101021008,11008,Karabar,8531.0
2,101021009,11009,Queanbeyan,11230.0
3,101021010,11010,Queanbeyan - East,4970.0
4,101021011,11011,Queanbeyan Region,17291.0


#### 1.2.2 Population Data 2021
Filter on only Interesting Columns

In [8]:
# Select 
cols = [
    'SA2_CODE_2021', 'SA2_5DIG21', 'SA2_CODE_2016',
    'SA2_5DIG16', 'SA2_NAME_2021', 'SA2_AREA',
    'ERP_P_202021', 'ERP_P_202021 (%)', 'ERP_212021'
]

df_2021 = df_2021[cols]

### 1.3 Quick Statistics

In [9]:
print(f"The 2016 data contains {len(df_2016)} different SA2s")
print(f"The 2021 data contains {len(df_2021)} different SA2s")

The 2016 data contains 2288 different SA2s
The 2021 data contains 2176 different SA2s


## 2. SA2 Codes
In previous preprocessing of the 2021 data, all SA2s that were changed from the 2016 SA2 codes (split) were removed from the 2021 data. Therefore, we can expect that there will be several codes in the 2016 data that will be missing in the 2021 data. This aligns with the fact that the 2021 data contains less rows than the 2016 data. We also expect that all SA2s in the 2021 data are contained in the 2016 data.

### 2.1 Remove SA2 Codes
We first exclude all SA2s from the 2016 data that were also excluded from the 2021 data.

In [10]:
# changed SA2 2016 codes
changed16_codes = [
    '212011291', '127011506', '206041126', '309071258', '115041302', '106021115', '309071257', '210051444', '316021420', '127031600', '125031482', '123021440',
    '116011308', '115011559', '101021011', '103011058', '106021117', '108041163', '116021310', '116021312', '117011322', '117021326', '117031332', '117031334',
    '117031335', '117031337', '117031338', '118011343', '118021348', '118021350', '119011357', '119011359', '119021363', '119021364', '119031368', '119031369',
    '119041375', '119041376', '120011384', '120021388', '120031390', '120031391', '120031575', '121011398', '121011400', '121011402', '121041415', '122021423', 
    '122031426', '122031428', '122031431', '123011434', '123011435', '123021442', '124031458', '125011473', '125021476', '125031485', '125041492', '126011495', 
    '126021497', '126021591', '127031598', '201011003', '201011004', '203021038', '203021041', '203031050', '204031074', '205041097', '206011105', '206011108', 
    '206021111', '206031114', '206041122', '206041123', '206051133', '206061137', '206071144', '207011151', '207011153', '209021207', '209021208', '209041219', 
    '209041430', '209041434', '210011229', '210031239', '210031438', '210041241', '210051249', '212021294', '212031301', '212031302', '212031305', '212041311', 
    '212041314', '212051320', '213011330', '213041355', '213041356', '213041357', '213051365', '213051366', '213051369', '213051465', '214021380', '216031417', 
    '301021527', '309071255', '309081261', '309101270', '310011273', '310041296', '311041320', '311041321', '313021365', '313051542', '314021388', '314021390', 
    '318021473', '402031037', '403041087', '501011002', '503021041', '504031059', '505021086', '505031103', '505031108', '506011113', '506021122', '507051185', 
    '801011112', '801101138'
]
# changed16_codes_5dig = [st[0]+st[-4:] for st in changed16_codes]
print(f"In total, there were {len(changed16_codes)} SA2s that had a change in 2021.")

In total, there were 134 SA2s that had a change in 2021.


In [11]:
# remove changed codes
len_df_2016 = len(df_2016)
df_2016 = df_2016[~df_2016["SA2_CODE_2016"].isin(changed16_codes)].reset_index(drop=True)
print(f"After excluding the SA2s that were also removed from the 2021 data, the 2016 data has {len(df_2016)} rows left (out of the {len_df_2016} rows) .")

After excluding the SA2s that were also removed from the 2021 data, the 2016 data has 2154 rows left (out of the 2288 rows) .


### 2.2 Check Assumptions
In the introduction of this section we made two assumptions, which we will test now.
#### 2.2.1 Assumption 1
We expect that there will be several codes in the 2016 data that are not present in the 2021 data. However, this should not be the case after excluding the SA2s that were also removed from the 2021 data.

In [12]:
for i,row in df_2016.iterrows():
    if row["SA2_CODE_2016"] not in list(df_2021["SA2_CODE_2016"]):
        print(i,row["SA2_NAME_2016"],row["NR_OF_PEOPLE_2016"])

773 Pakenham - South 29580.0


We find that there is one SA2 in the 2016 that is not present in the 2021 data. As we do not have data on this 2021 data of ```Pakenham - South``` in 2021, we will disregard this. 

In [13]:
print(len(df_2016))
df_2016 = df_2016[df_2016["SA2_NAME_2016"]!= "Pakenham - South"].reset_index(drop=True)
print(len(df_2016))

2154
2153


#### 2.2.2 Assumption 2
We expect that all SA2 codes in the 2021 data are contained in the 2016 data.

In [14]:
for i,row in df_2021.iterrows():
    if row["SA2_5DIG16"] not in list(df_2016["SA2_5DIG16"]):
        print(i,row["SA2_NAME_2021"],row["ERP_P_202021"])

519 Migratory - Offshore - Shipping (NSW) nan
520 No usual address (NSW) nan
934 Migratory - Offshore - Shipping (Vic.) nan
935 No usual address (Vic.) nan
1448 Migratory - Offshore - Shipping (Qld) nan
1449 No usual address (Qld) nan
1620 Migratory - Offshore - Shipping (SA) nan
1621 No usual address (SA) nan
1865 Migratory - Offshore - Shipping (WA) nan
1866 No usual address (WA) nan
1966 Migratory - Offshore - Shipping (Tas.) nan
1967 No usual address (Tas.) nan
2036 Migratory - Offshore - Shipping (NT) nan
2037 No usual address (NT) nan
2167 Migratory - Offshore - Shipping (ACT) nan
2168 No usual address (ACT) nan
2169 Christmas Island 1716.0
2170 Cocos (Keeling) Islands 602.0
2171 Jervis Bay 310.0
2172 Norfolk Island 2220.0
2173 Migratory - Offshore - Shipping (OT) nan
2174 No usual address (OT) nan
2175 Outside Australia nan


We find that there actually are several SA2s in the 2021 data that are not included in the 2016 data. When taking a closer look, we find that most occurrences have a 'NaN' value for the number of inhabitants (ERP_P_202021), and that most occurrences are two rows in the DataFrame after each other.

Furthermore, we often see the words 'Migratory' and 'No usual address'. The Australian Government <a href="https://www.abs.gov.au/ausstats/abs@.nsf/Lookup/by%20Subject/1270.0.55.001~July%202016~Main%20Features~Special%20purpose%20codes~10004">defines</a> these as:
- Migratory: code people who are in transit on long distance trains, buses, aircraft and long haul road transport vehicles on Census night.
- No Usual Address: code people with no fixed place of abode.

As the 2016 data disregards these, we will disregard this.

The rows that actually contain inhabitant data turn out to be places that are Australian external territory, meaning that they are away from the main land of Australia. This is however not the case for Jervis Bay, which seems to be an odd outlier. We can add <a href="https://www.abs.gov.au/census/find-census-data/quickstats/2016/901031003">Jervis Bay</a> to the 2016 data as follows: <!-- https://en.wikipedia.org/wiki/Jervis_Bay_Territory -->

In [15]:
jervis_bay = {'SA2_NAME_2016':['Jervis Bay'], 
              'SA2_5DIG16':['91003'], 
              'NR_OF_PEOPLE_2016':[391]
}

df_2016 = pd.concat([df_2016,pd.DataFrame(jervis_bay)], ignore_index=True)

## 3. Merge Datasets
Take the intersection of codes in both DataFrames and make sure the 2016 and 2021 DataFrames only contain these codes.

### 3.1 Get Relevant SA2 Codes
We find all SA2 2016 codes that we do not disregard, and filter df_2021 to only contain these codes.

In [16]:
lst_2016 = list(df_2016["SA2_5DIG16"])
lst_2021 = list(df_2021["SA2_5DIG16"])
SA2_codes = [code for code in lst_2016 if code in lst_2021]

df_2021 = df_2021[df_2021["SA2_5DIG16"].isin(SA2_codes)].reset_index(drop=True)

3.2 Merge Datasets

In [17]:
df_merged = pd.merge(df_2016, df_2021, on=['SA2_CODE_2016','SA2_5DIG16'], how='left')
df_merged.head()

,SA2_CODE_2016,SA2_5DIG16,SA2_NAME_2016,NR_OF_PEOPLE_2016,SA2_CODE_2021,SA2_5DIG21,SA2_NAME_2021,SA2_AREA,ERP_P_202021,ERP_P_202021 (%),ERP_212021
0,101021007,11007,Braidwood,3950.0,101021007,11007,Braidwood,3418.3525,4330.0,0.016856,1.3
1,101021008,11008,Karabar,8531.0,101021008,11008,Karabar,6.9825,8546.0,0.033268,1223.9
2,101021009,11009,Queanbeyan,11230.0,101021009,11009,Queanbeyan,4.7620,11370.0,0.044262,2387.7
3,101021010,11010,Queanbeyan - East,4970.0,101021010,11010,Queanbeyan - East,13.0032,5093.0,0.019826,391.7
4,101021012,11012,Queanbeyan West - Jerrabomberra,13150.0,101021012,11012,Queanbeyan West - Jerrabomberra,13.6748,12743.0,0.049607,931.9


Create Final Meta Data

In [18]:
# compute percentage per SA2
tot_nr_people2016 = 24.19e6 # on Google
df_merged["NR_OF_PEOPLE_2016_%"] = [(nr/tot_nr_people2016)*100 for nr in df_merged["NR_OF_PEOPLE_2016"]]

tot_nr_people2021 = 25.69e6 # on Google
df_merged["ERP_P_202021 (%)"] = [(nr/tot_nr_people2021)*100 for nr in df_merged["ERP_P_202021"]]

# Compute pop density for 2016
df_merged["POPULATION_DENSITY_2016"] = [row["NR_OF_PEOPLE_2016"]/row["SA2_AREA"] for _,row in df_merged.iterrows()]

# 4. Compare 2016 and 2021 Data
To see whether the changes are indeed minimal, we will look at two things. First, we will look at the differences in population, and second, we will look at the differences in population density. Here we make several hypotheses:
- the average increase in population per SA2 is almost the same as the average increase in population stated on Google
- the percentage of total Australian population per SA2 should be almost the same in the 2016 and 2021 data.
- the population density increases with the increase in population per SA2

## 4.1 Difference in Population
According to Google, the population of Australia increased from 24.19 million in 2016 to 25.69 million in 2021, an increase of 6.2%. Since we excluded several SA2s, our data does not contain all data, and therefore we expect to see an average increase that should be near 6.2% over all SA2s.

### 4.1.1 Change in Population per SA2

#### <font size=4> We first look at the <b>numerical difference</b> between the 2016 and 2021 population per SA2.</font>

In [19]:
df_merged["nr_diff_in_pop"] = [row["ERP_P_202021"] - row["NR_OF_PEOPLE_2016"] for _,row in df_merged.iterrows()]
df_merged["nr_diff_in_pop"].describe()

count     2117.000000
mean       488.458668
std       1249.720495
min      -5526.000000
25%        -28.000000
50%        209.000000
75%        646.000000
max      20151.000000
Name: nr_diff_in_pop, dtype: float64

In [20]:
fig = px.histogram(df_merged["nr_diff_in_pop"] ,width=700,nbins=300,range_x=[-2500, 8000])
fig.show()

In [21]:
df_merged[df_merged["nr_diff_in_pop"]>10000]

,SA2_CODE_2016,SA2_5DIG16,SA2_NAME_2016,NR_OF_PEOPLE_2016,SA2_CODE_2021,SA2_5DIG21,SA2_NAME_2021,SA2_AREA,ERP_P_202021,ERP_P_202021 (%),ERP_212021,NR_OF_PEOPLE_2016_%,POPULATION_DENSITY_2016,nr_diff_in_pop
769,212011287,21287,Beaconsfield - Officer,13612.0,212011546,21546,Beaconsfield - Officer,33.2503,23947.0,0.093215,720.2,0.056271,409.379765,10335.0
823,213041359,21359,Rockbank - Mount Cottrell,3148.0,213041359,21359,Rockbank - Mount Cottrell,125.7609,17836.0,0.069428,141.8,0.013014,25.031627,14688.0
891,209041437,21437,Wollert,9493.0,209041437,21437,Wollert,53.8920,24762.0,0.096388,459.5,0.039243,176.148593,15269.0
897,210051445,21445,Mickleham - Yuroke,3580.0,210051445,21445,Mickleham - Yuroke,70.8359,23731.0,0.092374,335.0,0.014800,50.539345,20151.0
919,213051468,21468,Werribee - West,12283.0,213051468,21468,Werribee - West,127.6631,22661.0,0.088209,177.5,0.050777,96.214176,10378.0


We find that the population per SA2 increased by almost 500 people per SA2. However, the 50th percentile shows an increase of around 200 people. This can also be seen in the histogram, where the increase in people per SA2 is centered around an increase between 0 and 99 people, indicating that there are some SA2s that have grown a lot. It turns out that only 5 SA2s have had an increase of more than 10.000 people, where only 1 was higher than 20.000.

#### <font size=4>We now look the <b>Percentual Difference</b> in populations. Since there are some SA2s that had a population of 0 in 2016, we cannot compute all changes. Because this is the case for only 1.6% of all SA2s, we exclude them from the analysis.

<b>Assumption:</b> We assumed that the average percentual increase in population per SA2 was near the average percentual increase of the total population, which is equal to ```6.2%```.</font>

In [22]:
df_merged_c = df_merged[df_merged["NR_OF_PEOPLE_2016"]!=0].copy()
df_merged_c["percentual_diff_in_pop"] = [((row["ERP_P_202021"] - row["NR_OF_PEOPLE_2016"])/row["NR_OF_PEOPLE_2016"])*100 for _,row in df_merged_c.iterrows()]
df_merged_c["percentual_diff_in_pop"].describe()

count    2107.000000
mean       14.300566
std       210.741465
min       -85.593220
25%        -0.543483
50%         2.970602
75%         7.499403
max      7668.000000
Name: percentual_diff_in_pop, dtype: float64

In [23]:
df_merged_c[df_merged_c["percentual_diff_in_pop"]>1000]

,SA2_CODE_2016,SA2_5DIG16,SA2_NAME_2016,NR_OF_PEOPLE_2016,SA2_CODE_2021,SA2_5DIG21,SA2_NAME_2021,SA2_AREA,ERP_P_202021,ERP_P_202021 (%),ERP_212021,NR_OF_PEOPLE_2016_%,POPULATION_DENSITY_2016,nr_diff_in_pop,percentual_diff_in_pop
296,117011320,11320,Banksmeadow,21.0,117011320,11320,Banksmeadow,3.9384,507.0,0.001974,128.7,0.000087,5.332115,486.0,2314.285714
635,206051131,21131,Port Melbourne Industrial,25.0,206051511,21511,Port Melbourne Industrial,7.6426,1942.0,0.007559,254.1,0.000103,3.271138,1917.0,7668.000000
1698,505021091,51091,Osborne Park Industrial,18.0,505021091,51091,Osborne Park Industrial,3.4452,229.0,0.000891,66.5,0.000074,5.224660,211.0,1172.222222
2041,801011018,81018,Lawson,146.0,801011018,81018,Lawson,3.0845,2746.0,0.010689,890.3,0.000604,47.333441,2600.0,1780.821918
2132,801041120,81120,Moncrieff,106.0,801041120,81120,Moncrieff,1.9172,5313.0,0.020681,2771.2,0.000438,55.288963,5207.0,4912.264151


We find that there are 5 outliers that have had an increase of more than 1000% in population. We will disregard these outliers, as we have removed many SA2s in earlier steps, and these can account for a inaccurate display in change.

In [24]:
df_merged_c[df_merged_c["percentual_diff_in_pop"]<1000]["percentual_diff_in_pop"].describe()

count    2102.000000
mean        5.843815
std        22.469995
min       -85.593220
25%        -0.547688
50%         2.960865
75%         7.462328
max       562.877095
Name: percentual_diff_in_pop, dtype: float64

In [25]:
fig = px.histogram(df_merged_c[df_merged_c["percentual_diff_in_pop"]<1000]["percentual_diff_in_pop"],width=700,nbins=400,range_x=[-100, 200])
fig.show()

For our subset, we find an average increase of more than 5.8%, which is very close to the average increase of 6.2% for the entire population. From the histogram we can deduce that the population of most SA2s has had an increase between 0 and 2%. We also see that the histogram is skewed to the right, accounting for the fact that the average percentual increase lies at around 5.8%.

### 4.1.2 Change in Percentage of Population with respect to the Total Population of Australia per SA2 !!

#### <font size=4>We first look at the <b>numerical difference</b> between the 2016 data and the 2021. 

<b>Assumptions:</b> We expect the change in percentage of population with respect to the total population of Australia, to be near ```0```.</font>

In [26]:
df_merged["nr_diff_pop_pctg"] = [row["NR_OF_PEOPLE_2016_%"]-row["ERP_P_202021 (%)"] for _,row in df_merged.iterrows()]
df_merged["nr_diff_pop_pctg"].describe()

count    2117.000000
mean        0.000472
std         0.004898
min        -0.077575
25%        -0.000278
50%         0.000777
75%         0.002285
max         0.024723
Name: nr_diff_pop_pctg, dtype: float64

In [44]:
fig = px.histogram(df_merged["nr_diff_pop_pctg"] ,width=1000,nbins=200,range_x=[-0.03, 0.02],title='Difference in Population with respect to the Total Population of Australia (%)')
fig.show()

As expected, we find that the average change in percentage of the population per SA2 is very near zero with a value of 0.000472.

#### <font size=4>We now look the <b>Percentual Difference</b> in the change in percentage of the population per SA2.</font>

In [28]:
df_merged["percentual_diff_pop_pctg"] = [(row["NR_OF_PEOPLE_2016_%"]-row["ERP_P_202021 (%)"])/row["ERP_P_202021 (%)"]*100 for _,row in df_merged.iterrows()]
df_merged["percentual_diff_pop_pctg"].describe()

count    2117.000000
mean        2.344086
std        24.601296
min      -100.000000
25%        -1.297887
50%         3.081643
75%         6.770886
max       637.159254
Name: percentual_diff_pop_pctg, dtype: float64

In [29]:
fig = px.histogram(df_merged["percentual_diff_pop_pctg"] ,width=700,range_x=[-50, 100],nbins=400)
fig.show()

We find an average increase of 2.3% and as can be seen in the histogram, the average percentual increase is very near 0.

## 4.2 Change in Population Density
Since the population density depends solely on the population of an SA2, and the area (in square KM) of an SA2, and the fact that the area of the SA2s in the data did not change, we expect to see an increase of around 6.2%, because this was also the case for the population.

#### <font size=4>We first look at the <b>numerical differences</b>.</font>

In [30]:
df_merged["nr_diff_pop_density"] = [row["ERP_212021"]-row["POPULATION_DENSITY_2016"] for _,row in df_merged.iterrows()]
df_merged["nr_diff_pop_density"].describe()

count    2076.000000
mean       35.836665
std       218.735642
min     -2958.479120
25%        -0.034675
50%         4.047945
75%        46.983468
max      3383.417108
Name: nr_diff_pop_density, dtype: float64

To get a more clear view of the actual distribution, we remove 19 outliers that have an absolute difference larger than 1000.

In [31]:
df_merged_d = df_merged[abs(df_merged["nr_diff_pop_density"])<1000].copy()
df_merged_d["nr_diff_pop_density"].describe()

count    2057.000000
mean       29.749848
std       123.883506
min      -976.732794
25%        -0.033980
50%         3.824925
75%        46.038401
max       952.973450
Name: nr_diff_pop_density, dtype: float64

In [32]:
fig = px.histogram(df_merged_d["nr_diff_pop_density"] ,width=700,nbins=100,range_x=[-500, 500])#,nbins=3000)
fig.show()

Without the outliers, we find that the population density of most SA2s increased by 0 to 20.

#### <font size=4>Now we look at the <b>Percentual Differences</b>.

<b>Assumption:</b> We expect that the average population density has grown with the average population, and therefore we expect an increase of around ```6.2%``` </font>

We again remove all rows with a population density of 0 in 2016, because this results in a population density of 0.

In [33]:
df_merged_e = df_merged[df_merged["POPULATION_DENSITY_2016"]!=0].copy()
df_merged_e["percentual_diff_pop_density"] = [(row["ERP_212021"]-row["POPULATION_DENSITY_2016"])/row["POPULATION_DENSITY_2016"]*100 for _,row in df_merged_e.iterrows()]
df_merged_e["percentual_diff_pop_density"].describe()

count    2068.000000
mean       14.936737
std       212.739434
min       -85.485653
25%        -0.427747
50%         3.259120
75%         7.931230
max      7667.938640
Name: percentual_diff_pop_density, dtype: float64

We again remove the outliers.

In [34]:
df_merged_e[df_merged_e["percentual_diff_pop_density"]<300]["percentual_diff_pop_density"].describe()

count    2061.000000
mean        5.828379
std        16.751357
min       -85.485653
25%        -0.457326
50%         3.236084
75%         7.803525
max       208.398128
Name: percentual_diff_pop_density, dtype: float64

In [35]:
fig = px.histogram(df_merged_e[df_merged_e["percentual_diff_pop_density"]<300]["percentual_diff_pop_density"] ,width=700,nbins=100)#,range_x=[-500, 500])
fig.show()

We see an identical result as with the percentual change in population.

# 5. Summary

The client wanted to have the most recent data that was available. For the population data, this meant the use of 2021 population data. While the other data is from 2021, it is still possible to use the 2021 data, as the differences are very slim. According to Google, the population of Australia increased from 24.19 million in 2016 to 25.69 million in 2021, an increase of 6.2%. After comparing the 2016 data with the 2021 data, it is found that the average increase in population per SA2 is equal to 5.8%, which is consistent with the increase in population density from 2016 to 2021 in our data. This value is near the real-world value, and since several SA2s were excluded (see section \ref{}), the 2021 data contains only a subset of all data. This accounts for the small difference. Moreover, we find that the change in percentage of the population with respect to the total population of Australia per SA2 is very close to zero, with an average difference of 0.000472. 